In [14]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)

    #dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file"]
    #duplicates = df_baseline[df_baseline.duplicated(subset=dup_columns)].index.to_list()
    #df_baseline = df_baseline.drop_duplicates(subset=dup_columns, keep="first")
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/validation_set_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []
        skipped = 0

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            #if index in duplicates:
            #    continue
            
            response_dict = None
            isDependency = None

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response, strict=False)
                isDependency = response_dict["isDependency"]
            except (json.JSONDecodeError, KeyError):
                skipped += 1
                continue
                
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print(f"Num Failures: {fp + fn}")

        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))
        print("Entries skipped: ", skipped)

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness_{index_name}.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/results/validation_set.csv",
    model_names=["gpt-4o-2024-05-13", "gpt-3.5-turbo-0125", "llama3:70b", "llama3:8b"], #"llama3.1:70b"
    result_dir="../data/results/validation",
    index_name="all"
)

Num Failures: 13
Model:  gpt-4o-2024-05-13
File:  ../data/results/validation/validation_set_without_gpt-4o-2024-05-13.json
TP 20
FP 2
TN 17
FN 11
Sum correct response format:  50
Entries skipped:  0
Accuracy 0.74
Precision 0.9090909090909091
Recall 0.6451612903225806
F1 Score:  0.7547169811320754
Num Failures: 19
Model:  gpt-3.5-turbo-0125
File:  ../data/results/validation/validation_set_without_gpt-3.5-turbo-0125.json
TP 19
FP 7
TN 12
FN 12
Sum correct response format:  50
Entries skipped:  0
Accuracy 0.62
Precision 0.7307692307692307
Recall 0.6129032258064516
F1 Score:  0.6666666666666667
Num Failures: 19
Model:  llama3:70b
File:  ../data/results/validation/validation_set_without_llama3:70b.json
TP 18
FP 6
TN 13
FN 13
Sum correct response format:  50
Entries skipped:  0
Accuracy 0.62
Precision 0.75
Recall 0.5806451612903226
F1 Score:  0.6545454545454547
Num Failures: 15
Model:  llama3:8b
File:  ../data/results/validation/validation_set_without_llama3:8b.json
TP 21
FP 12
TN 4
FN 3
Sum

In [15]:
import pandas as pd


file_path = "../data/results/validation/validation_effectiveness_all.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-4o-2024-05-13,../data/results/validation/validation_set_with...,0.740,0.909091,0.645161,0.754717
1,gpt-3.5-turbo-0125,../data/results/validation/validation_set_with...,0.620,0.730769,0.612903,0.666667
2,llama3:70b,../data/results/validation/validation_set_with...,0.620,0.750000,0.580645,0.654545
3,llama3:8b,../data/results/validation/validation_set_with...,0.625,0.636364,0.875000,0.736842
